Data Processing

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler

In [2]:
# Load the dataset
data = pd.read_csv('merged_data.csv')

# Display the first few rows to inspect the structure
print(data.head())

# Show basic information about the dataset
print(data.info())

# Summarize numerical columns with descriptive statistics
print(data.describe())

# Check for missing or invalid values
print(data.isnull().sum())  # Count of NaN values in each column

      #YY   MM   DD   hh    mm  WDIR  WSPD  GST  WVHT   DPD  ...    PRES  \
0     NaN  NaN  NaN  NaN   NaN   NaN   NaN  NaN   NaN   NaN  ...     NaN   
1  2020.0  1.0  1.0  0.0   0.0  69.0   5.5  7.1   1.3  14.3  ...  1014.4   
2  2020.0  1.0  1.0  0.0  10.0  64.0   4.5  5.5  99.0  99.0  ...  1014.4   
3  2020.0  1.0  1.0  0.0  20.0  63.0   4.7  5.8  99.0  99.0  ...  1014.5   
4  2020.0  1.0  1.0  0.0  30.0  72.0   5.1  6.4  99.0  99.0  ...  1014.6   

   ATMP   WTMP   DEWP   VIS  TIDE  WDI  R WSP  D GST  WSP  
0   NaN    NaN    NaN   NaN   NaN  NaN    NaN    NaN  NaN  
1  26.7   28.1  999.0  99.0  99.0  NaN    NaN    NaN  NaN  
2  26.6  999.0  999.0  99.0  99.0  NaN    NaN    NaN  NaN  
3  26.9  999.0  999.0  99.0  99.0  NaN    NaN    NaN  NaN  
4  27.0  999.0  999.0  99.0  99.0  NaN    NaN    NaN  NaN  

[5 rows x 22 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206920 entries, 0 to 206919
Data columns (total 22 columns):
 #   Column  Non-Null Count   Dtype  
---  ------

In [3]:
columns_to_keep = ['#YY', 'MM', 'DD', 'ATMP', 'WTMP', 'WVHT']
data = data[columns_to_keep]
# Drop the first row from the DataFrame
data = data.iloc[1:].reset_index(drop=True)
# Display the DataFrame to verify
print(data.head())

      #YY   MM   DD  ATMP   WTMP  WVHT
0  2020.0  1.0  1.0  26.7   28.1   1.3
1  2020.0  1.0  1.0  26.6  999.0  99.0
2  2020.0  1.0  1.0  26.9  999.0  99.0
3  2020.0  1.0  1.0  27.0  999.0  99.0
4  2020.0  1.0  1.0  27.0  999.0  99.0


Fill NA and extreme value

In [4]:
def fill_invalid_values(data, column, invalid_values):
    # Combine '#YY', 'MM', 'DD' into a single date column
    data['date'] = pd.to_datetime(
        dict(year=data['#YY'], month=data['MM'], day=data['DD']),
        errors='coerce'
    )

    # Ensure there are no invalid or missing dates
    if data['date'].isnull().any():
        print("Warning: Some dates could not be parsed. Filling with nearby dates.")
        data['date'] = data['date'].fillna(method='ffill').fillna(method='bfill')

    # Replace invalid values (999, 99) with NaN for processing
    data[column] = data[column].replace(invalid_values, np.nan)

    # Group by 'date' to handle data at the daily level
    grouped = data.groupby('date')

    def process_group(group):
        # If all values in the group are NaN, return as-is (handled later by interpolation)
        if group[column].isnull().all():
            return group

        # Fill NaN within the day using the daily mean
        group[column] = group[column].fillna(group[column].mean())
        return group

    # Apply the processing function to each group
    data = grouped.apply(process_group).reset_index(drop=True)

    # Handle days where all values are NaN
    def interpolate_missing_days(data, column):
        # Calculate daily means, including NaN for fully missing days
        daily_mean = data.groupby('date')[column].mean()

        # Interpolate missing daily means linearly
        interpolated_mean = daily_mean.interpolate(method='linear', limit_direction='both')

        # Map interpolated daily means back to the original DataFrame
        data[column] = data['date'].map(interpolated_mean)
        return data

    # Interpolate missing days for completely missing data
    data = interpolate_missing_days(data, column)

    return data

In [5]:
# Replace invalid values for each column
data = fill_invalid_values(data, 'WTMP', invalid_values=[999, 99])
data = fill_invalid_values(data, 'ATMP', invalid_values=[999, 99])
data = fill_invalid_values(data, 'WVHT', invalid_values=[999, 99])
data

/var/folders/j6/gnlkxfsx0mz7c1ngs9mbs16m0000gn/T/ipykernel_70832/2395899682.py:11: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data['date'] = data['date'].fillna(method='ffill').fillna(method='bfill')
/var/folders/j6/gnlkxfsx0mz7c1ngs9mbs16m0000gn/T/ipykernel_70832/2395899682.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data = grouped.apply(process_group).reset_index(drop=True)
/var/folders/j6/gnlkxfsx0mz7c1ngs9mbs16m0000gn/T/ipykernel_70832/2395899682.py:11: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data['d

/var/folders/j6/gnlkxfsx0mz7c1ngs9mbs16m0000gn/T/ipykernel_70832/2395899682.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data = grouped.apply(process_group).reset_index(drop=True)
/var/folders/j6/gnlkxfsx0mz7c1ngs9mbs16m0000gn/T/ipykernel_70832/2395899682.py:11: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data['date'] = data['date'].fillna(method='ffill').fillna(method='bfill')
/var/folders/j6/gnlkxfsx0mz7c1ngs9mbs16m0000gn/T/ipykernel_70832/2395899682.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas th

,#YY,MM,DD,ATMP,WTMP,WVHT,date
0,2020.0,1.0,1.0,26.772222,28.212500,1.245833,2020-01-01
1,2020.0,1.0,1.0,26.772222,28.212500,1.245833,2020-01-01
2,2020.0,1.0,1.0,26.772222,28.212500,1.245833,2020-01-01
3,2020.0,1.0,1.0,26.772222,28.212500,1.245833,2020-01-01
4,2020.0,1.0,1.0,26.772222,28.212500,1.245833,2020-01-01
...,...,...,...,...,...,...,...
206914,2023.0,12.0,31.0,25.808451,27.595833,1.254167,2023-12-31
206915,2023.0,12.0,31.0,25.808451,27.595833,1.254167,2023-12-31
206916,2023.0,12.0,31.0,25.808451,27.595833,1.254167,2023-12-31
206917,2023.0,12.0,31.0,25.808451,27.595833,1.254167,2023-12-31


In [8]:
#Check for nan 
print(data.isnull().sum())
data = data.dropna(subset=['#YY', 'MM', 'DD'])
print("After dropping NaN:")
print(data.isnull().sum())

#YY     3
MM      3
DD      3
ATMP    0
WTMP    0
WVHT    0
date    0
dtype: int64
After dropping NaN:
#YY     0
MM      0
DD      0
ATMP    0
WTMP    0
WVHT    0
date    0
dtype: int64


In [10]:
def keep_max_wvht_per_day(data):
    """
    Keep only the row with the maximum WVHT value for each day and drop the rest.

    Args:
        data (pd.DataFrame): Input dataset with '#YY', 'MM', 'DD', and 'WVHT' columns.

    Returns:
        pd.DataFrame: Dataset with only one row per day, corresponding to the maximum WVHT.
    """
    # Combine '#YY', 'MM', 'DD' into a single date column
    data['date'] = pd.to_datetime(
        dict(year=data['#YY'], month=data['MM'], day=data['DD']),
        errors='coerce'
    )

    # Ensure that the 'date' column was created successfully
    if data['date'].isnull().any():
        print("Warning: Some dates could not be parsed. Dropping invalid rows.")
        data = data.dropna(subset=['date'])

    # Group by 'date' and find the row with the maximum WVHT for each day
    max_wvht_data = (
        data.loc[data.groupby('date')['WVHT'].idxmax()]
        .reset_index(drop=True)
    )

    # Drop the temporary 'date' column if not needed
    max_wvht_data = max_wvht_data.drop(columns=['date'])

    return max_wvht_data

data = keep_max_wvht_per_day(data)
data

,#YY,MM,DD,ATMP,WTMP,WVHT
0,2020.0,1.0,1.0,26.772222,28.212500,1.245833
1,2020.0,1.0,2.0,27.189928,28.054167,1.108333
2,2020.0,1.0,3.0,26.284722,27.933333,1.212500
3,2020.0,1.0,4.0,26.873611,27.754167,1.291667
4,2020.0,1.0,5.0,25.072222,27.808333,1.233333
...,...,...,...,...,...,...
1448,2023.0,12.0,27.0,25.494118,27.762500,1.295833
1449,2023.0,12.0,28.0,25.529167,27.775000,1.337500
1450,2023.0,12.0,29.0,25.337324,27.816667,1.183333
1451,2023.0,12.0,30.0,25.756643,27.658333,1.091667


In [11]:
data

,#YY,MM,DD,ATMP,WTMP,WVHT
0,2020.0,1.0,1.0,26.772222,28.212500,1.245833
1,2020.0,1.0,2.0,27.189928,28.054167,1.108333
2,2020.0,1.0,3.0,26.284722,27.933333,1.212500
3,2020.0,1.0,4.0,26.873611,27.754167,1.291667
4,2020.0,1.0,5.0,25.072222,27.808333,1.233333
...,...,...,...,...,...,...
1448,2023.0,12.0,27.0,25.494118,27.762500,1.295833
1449,2023.0,12.0,28.0,25.529167,27.775000,1.337500
1450,2023.0,12.0,29.0,25.337324,27.816667,1.183333
1451,2023.0,12.0,30.0,25.756643,27.658333,1.091667


Normalize data

In [12]:
# Select columns to normalize
columns_to_normalize = ['ATMP', 'WTMP', 'WVHT']
data_to_normalize = data[columns_to_normalize]

# Initialize MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform the data
normalized_data = scaler.fit_transform(data_to_normalize)

# Replace original columns with normalized values
data_normalized = data.copy()
data_normalized[columns_to_normalize] = normalized_data

# Verify the normalization
print(data_normalized.head())

      #YY   MM   DD      ATMP      WTMP      WVHT
0  2020.0  1.0  1.0  0.395523  0.521815  0.367322
1  2020.0  1.0  2.0  0.445842  0.488656  0.326781
2  2020.0  1.0  3.0  0.336797  0.463351  0.357494
3  2020.0  1.0  4.0  0.407737  0.425829  0.380835
4  2020.0  1.0  5.0  0.190735  0.437173  0.363636


Select only Time directly related features

In [13]:
# 4. to CSV
output_file = "merged_TimeWVHT_data.csv"
data.to_csv(output_file, index=False)

data = pd.read_csv("merged_TimeWVHT_data.csv")